In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import quantstats as qs
import yfinance as yf
from hurst import compute_Hc 
from modules.strategy import group4_ensemble_model_signals

from zipline.api import (order, order_target, record, symbol, get_datetime,
                         order_target_percent,
                         order_target_value,
                         set_benchmark,
                         get_open_orders)
from zipline import run_algorithm

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
tickers = os.listdir('./CSVs/daily/')
tickers = [x.replace('.csv', '') for x in tickers]

In [3]:
start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2022-12-31')

window = 100
capital_base = 100000
position = 5

In [4]:
%%capture backtest_cell_output 

def initialize(context):
    context.i = 0

    assets = list()
    for ticker in tickers:
        assets.append(symbol(ticker))
    context.assets = assets
    
    
def handle_data(context, data):
    context.i += 1
    if context.i < window + 1:
        return

    for asset in context.assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            cash = context.portfolio.cash
            price = data.current(asset, "price")

            curr_win_data = data.history(asset, ['open', 'high', 'low', 'close', 'volume'], bar_count=window, frequency="1d")
            signal = group4_ensemble_model_signals(curr_win_data)

            if signal == 1:
                order_target(asset, 5)
            elif signal == -1: 
                order_target(asset, 0)
            else:
                pass
    


result = run_algorithm(
    start=pd.to_datetime(start_date),
    end=pd.to_datetime(end_date),
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    bundle='snp50',
    data_frequency='daily',
    )
                        

ValueError: Series contains NaNs

In [7]:
for ticker in tickers:
    df = pd.read_csv(f'CSVs/daily/{ticker}.csv')
    df.set_index('date', inplace=True)
    if df.close.isnull().values.any():
        print(ticker)


In [ ]:
benchmark_returns = yf.download('^sp100', start=start_date, end=end_date)['Adj Close'].pct_change()

qs.reports.full(result.returns, benchmark_rets=benchmark_returns, df=result)